In [ ]:
def custom_RandomForestRegressor(data, target_col, dependent_col, val_data=None):
    data['val'] = 0
    
    if val_data is not None:
        val_data['val'] = 1
        data = data.append(val_data)
    
    X = data[dependent_col + ['val']]
    y = data[[target_col, 'val']]

    onehot_col = []

    for col in X.columns:
        if isinstance(X[col].values[0], str):
            onehot_col.append(col)

    X = pd.get_dummies(X, prefix=onehot_col)
    
    if val_data is not None:
        val_X = X[X['val'] == 1]
        val_y = y[y['val'] == 1]
        X = X[X['val'] == 0]
        y = y[y['val'] == 0]
        
        val_X = val_X.drop(['val'], axis=1)
        val_y = val_y.drop(['val'], axis=1)

    X = X.drop(['val'], axis=1)
    y = y.drop(['val'], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

    # model train

    reg = RandomForestRegressor(n_estimators=20, max_depth=5, random_state=0)
    reg.fit(X, y)

    X_pred = reg.predict(X_test)
    X_pred = [round(x) for x in X_pred]

    print('MSE : {}'.format(mean_squared_error(y_test, X_pred)))
    print('Offical Numeric Score : {}'.format(official_numerical_score(y_test.values, X_pred)))
    
    if val_data is not None:
        val_X_pred = reg.predict(val_X)
        val_X_pred = [round(x) for x in val_X_pred]
        print('Offical Numeric Validation Score : {}'.format(official_numerical_score(val_y.values, val_X_pred)))

def custom_RandomForestClassifier(data, target_col, dependent_col, val_data=None):
    data['val'] = 0
    
    if val_data is not None:
        val_data['val'] = 1
        data = data.append(val_data)
    
    X = data[dependent_col + ['val']]
    y = data[[target_col, 'val']]

    onehot_col = []

    for col in X.columns:
        if isinstance(X[col].values[0], str):
            onehot_col.append(col)

    X = pd.get_dummies(X, prefix=onehot_col)
    
    if val_data is not None:
        val_X = X[X['val'] == 1]
        val_y = y[y['val'] == 1]
        X = X[X['val'] == 0]
        y = y[y['val'] == 0]
        
        val_X = val_X.drop(['val'], axis=1)
        val_y = val_y.drop(['val'], axis=1)

    X = X.drop(['val'], axis=1)
    y = y.drop(['val'], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

    # model train

    clf = RandomForestClassifier(n_estimators=20, max_depth=5, random_state=0)
    clf.fit(X, y)

    X_pred = clf.predict(X_test)

    print('acc : {}'.format(accuracy_score(y_test, X_pred)))
    print('Offical Categorical Score : {}'.format(official_categorical_score(y_test.values, X_pred)))
    
    if val_data is not None:
        val_X_pred = clf.predict(val_X)
        print('Offical Categorical Validation Score : {}'.format(official_categorical_score(val_y.values, val_X_pred)))
    
meaningful_dependent_col = ['사망자수', '사상자수', '중상자수', '경상자수', '부상신고자수',
                            '발생년', '발생년월일시', '발생분', '주야', '요일', '발생지시도', '발생지시군구',
                            '사고유형_대분류', '사고유형_중분류', '사고유형', '법규위반_대분류', '법규위반', '도로형태_대분류', '도로형태', 
                            '당사자종별_1당_대분류', '당사자종별_1당', '당사자종별_2당_대분류', '당사자종별_2당']

In [ ]:
for target_col in target_col_list:
    print(target_col)
    temp_col = meaningful_dependent_col.copy()
    temp_col.remove(target_col)
    if isinstance(train_data[target_col].values[0], str):
        custom_RandomForestClassifier(train_data, target_col=target_col, dependent_col=temp_col, val_data=val_data)
    else:
        custom_RandomForestRegressor(train_data, target_col=target_col, dependent_col=temp_col, val_data=val_data)
    print()

In [ ]:
model_dict = {}

for i, row in test.iterrows():
    temp = []
    for key, value in row.items():
        if isinstance(value, float) and np.isnan(value):
            temp.append(key)
    target_col_string = '/'.join(temp)
    model_dict[target_col_string] = []